# 👩‍💻 BBC CUSTOMER ANALYSIS PROJECT
---

# INTRODUCTION
The project aims to know if BBC customers in different BBC ages have different serviceid cluster, and how we do cross-sale to customers. 

## Objectives
1. The first goal of this project is to use SQL to define: 
    - The first 2 serviceid customers used and Date that customer used those services.
    - The last serviceid users used and Date that users used that service.
    - The total distinct serviceid that customers used.
    - Present the findings into one table with the columns as follows:
 
    
| User_id  | FirstServiceid | SecondServiceid | FirstServiceDate | SecondServiceDate | LastServiceid | LastServiceDate | TotalService |
| -------- | -------------- | --------------- | ---------------- | ----------------- | ------------- | --------------- | ------------ |
| 407901   | 12  |  45 | 1-Jan-18 | 2-Jan-18 | 667 | 22-July-2018| 6 |
    
   
    
2. The second goal is identifying patterns using Python to answer the business questions below.

## Business question
1. Determine whether customers in different BBC ages have different serviceid cluster?
2. Determine which serviceid we can cross-sell to customers? Give recommendation to your findings.

## Database description
We have one table `test` with three columns `User_id`, `Date`, and `Serviceid`.

- **User_id**: The identity of the individual customer.
- **Servicedid**: The identity of the transaction.
- **Date**: the date that transaction is performed.

_Note: This is the open data._

# PROCESS
Doing SQL query to find the answer for the first goal, we use the combination of three `CTE`s to find the answer for `The first 2 serviceid`, `last serviceid` csutomers used and `Date` 

Combine with `COUNT` statement to find the total distinct serviceid that customers used. Then join everything to show the results as the required table above.

Natural join is being used to optimize the time efficiency. No JOIN command, but it is implicit (INNER JOIN).

```sql
-- create a ranked_cte to sort data with ROW_NUMBER
WITH ranked_cte AS( 
SELECT User_Id, date, serviceid,
ROW_NUMBER() OVER(PARTITION BY User_Id
ORDER BY Date) RN,
ROW_NUMBER() OVER(PARTITION BY User_Id
ORDER BY Date DESC) NR
FROM test
),
-- create the first_cte to define the firstservice value by choosing the RN = 1
first_cte AS(
SELECT t.User_Id, r.serviceid AS FirstServiceid, r.date AS FirstServiceDate
FROM ranked_cte AS r, test AS t
WHERE RN = 1 
AND r.User_Id = t.User_Id
GROUP BY t.User_Id
),
-- create second_cte to define the secondservice value by choosing RN = 2
second_cte AS(
SELECT f.*, r.serviceid as SecondServiceid, r.date AS SecondServiceDate
FROM first_cte as f, 
	 ranked_cte as r
WHERE RN = 2
AND r.User_Id = f.User_Id
GROUP BY r.User_Id
) 
-- Join everything, use the DATE_FORMAT statement to convert the date into your desired format
SELECT t.User_Id AS User_id, s.FirstServiceid, 
    s.SecondServiceid, DATE_FORMAT(s.FirstServiceDate, "%e" "-" "%b" "-" "%y") AS FirstServiceDate,
    DATE_FORMAT(s.SecondServiceDate, "%e" "-" "%b" "-" "%y") AS SecondServiceDate,
    r.serviceid AS LastServiceid, DATE_FORMAT(r.date, "%e" "-" "%b" "-" "%y") AS LastServiceDate,
    COUNT(DISTINCT t.serviceid) AS TotalService -- find the total distinct serviceid each customer used
FROM second_cte AS s,
     ranked_cte AS r,
     test AS t
WHERE NR = 1
AND s.User_Id = r.User_Id
AND t.User_Id = r.User_Id
GROUP BY r.User_Id;
;
```

Now that we have all the data in a single result table let's download it as a `CSV` file.

Next step, using `Python` to prepare and process the data.

# Importing Libraries


In [1]:
# Data processing
import numpy as np # data arrays
import pandas as pd # data structure and data analysis
import functools
import datetime as dt # date time


from mlxtend.frequent_patterns import apriori # Data pattern exploration
from mlxtend.frequent_patterns import association_rules # Association rules conversion

# Data Exploration
File "BBC_analysis.csv" has been extracted by querying from dataset "User_data.csv" contains info for further analysis. Below, you can see a sample of this info. 

In [2]:
# Read in data in CSV format
df1 = pd.read_csv('../input/bbc-customer-analysis-personal-project/BBC_analysis.csv')
df1.head() 

In [3]:
df1.info() # print to check the dtype

**From the summary, we can see that**

- There are a total of 43 rows and 7 columns in the data set.
- Data types are assigned correctly (apart from Servicedate). We will attempt at converting the datatype to a form suitable for analysis in the next section.
- The are no null values.

In [4]:
# Read in data in CSV format
df2 = pd.read_csv('../input/bbc-customer-analysis-personal-project/User_data.csv')
df2.head()

In [5]:
df2.info()

**From the summary, we can see that**

- There are a total of 1464 rows and 3 columns in the data set.
- Data types are assigned correctly (apart from Date). We will attempt at converting the datatype to a form suitable for analysis in the next section.
- The are no null values.

In [6]:
# Rename the column to match with above table
df2.rename(columns = {'User_Id':'User_id'}, inplace=True)
df2.head()

## 🚩 Analysis question 1

We would like to know whether customers in different BBC ages have different serviceid cluster?

First, we attempt to merge the two tables to see a big picture including the serviceid and the day of service transaction of each customer. In addition, this will help categorize customer into Ages groups and count service transactions for each group. 

In [7]:
#Merge df1 and df2 into one datafram dfs for further analysis
dfs = [df1, df2]
UserTable = functools.reduce(lambda left,right: pd.merge(left,right,on='User_id', how='outer'), dfs)
UserTable.dropna(inplace=True)
UserTable.head() # print to confirm 

**Converting** data into dtype datatime64 to define the correct date for next section, then print the info to double check.

In [8]:
#convert all service date to datatime64 dtype
UserTable['FirstServiceDate'] = pd.to_datetime(UserTable['FirstServiceDate'])
UserTable['SecondServiceDate'] = pd.to_datetime(UserTable['SecondServiceDate'])
UserTable['LastServiceDate'] = pd.to_datetime(UserTable['LastServiceDate'])
UserTable['Date'] = pd.to_datetime(UserTable['Date'])

# re-print information to confirm
UserTable.info()

# print 1st 5 rows to confirm
UserTable.head()

**Calculate the User_age**

To know if different BBC customer ages convert into differents serviceid cluster, we will count the `User_age` first then arrange them into the coresponding `Age_group`. Let's see the results as follows

In [9]:
#Calculate the User_age
UserTable['User_age'] = 1 + (UserTable['LastServiceDate']-UserTable['FirstServiceDate']).dt.days

# Split users into 5 categories of 30_age, 100_age, 200_age, 300_age, 300_plus_age
bins = [-1, 30, 100, 200, 300, np.inf]
names = ['30_age', '100_age', '200_age', '300_age', '300_plus_age']
UserTable['Age_bin'] = pd.cut(UserTable['User_age'], bins, labels = names)

#Print the first 10 rows to confirm
UserTable.head(10)


This step is to **group** and **count** customers bases on categories of BBC Age_bin and servicedid in order to know which services were used most by each group, then sort to see the first 5 highest values describing number of BBC customers per serviceid in each Age_group.

In [10]:
UserAgg = UserTable.groupby(['Age_bin','Serviceid'])[['User_id']].count().dropna()
UserAgg

#We group by the first level of the index:
g = UserAgg['User_id'].groupby(level=0, group_keys=False)

# Sort 'order' each group and take the first five elements:
g.nlargest(5)


### **Observation**

From the above analysis, we can see that:
- Customers in 30_age group tend to use Serviceid of 487, 1014, 299, 47, 65.
- Customers in 100_age group tend to use Serviceid of 333, 667, 981, 1014, 487.
- Customers in 200_age group tend to use Serviceid of 981, 20, 1014, 271, 326.
- Customers in 300_age group tend to use Serviceid of 18, 667, 333, 981, 268.
- Customers in 300_plus_age group tend to use Serviceid of 981, 19, 1014, 2, 271.


## 🚩 Analysis question 2:
We would like to know which serviceid we can cross-sales to users?

First, what is cross-selling? 
Cross-selling is the practice of selling related or additional products to existing customers. To know which `serviceid`s we can cross-sell for customer, we need to find which items (serviceid) that customers usually buy together then combine these items into combo.

The analysis below is based on the [Assocations Rules](https://towardsdatascience.com/association-rules-2-aa9a77241654) 
 and [Apriori principle](https://towardsdatascience.com/complete-guide-to-association-rules-2-2-c92072b56c84).
 
The Rules do not extract an individual’s preference, rather find **relationships between set of elements of every distinct transaction**. 

In [11]:
# Present the UserTable again to find pattern for cross-sell analysis
UserTable.head()


**Create** the df cross_sell for further analysis & **Find** the association between unique serviceids per day and each User_id.

In [12]:
#create the df cross_sell for further analysis  
Cross_sell = pd.DataFrame(UserTable, columns =['User_id', 'Date', 'Serviceid']) 

#Find the association between unique serviceids per day and each User_id
Cross_sell['User_id & Date'] = Cross_sell['User_id'].astype('str') + ' ' + Cross_sell['Date'].astype('str')
Cross_sell.drop(['User_id', 'Date'], axis=1, inplace=True) 
Cross_sell.head(10) #print the first ten rows to check 

Use `drop_duplicates` function to eliminate duplicate rows. For example, a customer use 1 particular serviceid twice a day, all we need is to know the number of unique serviceid per day for each customer.

Another example, if a customer usually buy razer along with blades, we can cross-sell 2 items in a combo.

In [13]:
Cross_sell.drop_duplicates(inplace = True)
Cross_sell.set_index('User_id & Date', inplace=True)
Cross_sell.head(10) #print the first ten rows to check 

**Convert** categorical serviceid to numeric through get_dummies to see the customers per day buy which `serviceid`s.

In [14]:
Cross_sell['Serviceid'] = Cross_sell['Serviceid'].astype('str')

basket = pd.get_dummies(Cross_sell)
basket.head()

**Convert** the service baskets into pivot table type to see the total.

In [15]:
basket_sets = pd.pivot_table(basket, index='User_id & Date', aggfunc='sum')
basket_sets

**Create** the `frequent_itemsets` through apriori and add a new column that stores the length of each itemset. Frequent itemsets are the ones which occur at least a minimum number of times in the transactions. Technically, these are the itemsets for which support value (fraction of transactions containing the itemset) is above a minimum threshold.

Take this as an example, the number of transactions containing items {Bread, Egg} is greater than or equal to number of transactions containing {Bread, Egg, Vegetables}. If the latter occurs in 30 transactions, former is occurring in all 30 of them and possibly will occur in even some more transactions. So if support value of {Bread, Egg, Vegetables} i.e. (30/100) = 0.3 is above minsup, then we can be assured that support value of {Bread, Egg} i.e. (>30/100) = >0.3 is above minsup too. 

Get support [here](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)

In [16]:
frequent_itemsets = apriori(basket_sets, min_support = 0.03, use_colnames = True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

Then, we can customize the results that satisfy our desired criteria as follows. In this case we put the `length` > 1 (meaning that there are more than one item in the item_sets and the `support` >=0.02. 

In [17]:
frequent_itemsets[ (frequent_itemsets['length'] > 1) &
                   (frequent_itemsets['support'] >= 0.02)]

### Generating the association rule: 
- Association Rule: Ex. {X → Y} is a representation of finding Y on the basket which has X on it
- Itemset: Ex. {X,Y} is a representation of the list of all items which form the association rule
- Support: Fraction of transactions containing the itemset
- Confidence:  Measures how often items in Y appear in transactions that contain X.
- Lift: Ratio of confidence to baseline probability of occurrence of {Y}

**Selecting** the important parameters for analysis

In [18]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('support', ascending=False).head(10)

## 📌 Observations:
- If the customer buy serviceid_667, we will cross-sell the serviceid_333 with the Support of 19%, Confidence ~ 92% and Lift >1.
- If the customer buy serviceid_333, we will cross-sell the serviceid_667 with the Support of 19%, Confidence ~ 99% and Lift >1.
- If the customer buy serviceid_981, we will cross-sell the serviceid_1014 with the Support of 17%, Confidence ~ 62% and Lift >1.
- If the customer buy serviceid_1014, we will cross-sell the serviceid_268 with the Support of 17%, Confidence ~ 79% and Lift >1.
- If the customer buy serviceid_982, we will cross-sell the serviceid_268 with the Support of 11%, Confidence ~ 98% and Lift >1.

...

- If the customer buy serviceid_326, we will cross-sell the serviceid_666 with the Support of 10%, Confidence of 100% and Lift >1.


# REFERENCES
- [Pandas Groupby](https://stackoverflow.com/questions/27842613/pandas-groupby-sort-within-groups)
- [Association Rules](https://towardsdatascience.com/association-rules-2-aa9a77241654)
- [Apriori Algorithm](https://towardsdatascience.com/complete-guide-to-association-rules-2-2-c92072b56c84)
- [Cross-sell](https://www.youtube.com/watch?v=VMavY0pBo2o&ab_channel=AngossSoftware)
- [Frequent Itemsets via Apriori Algorithm](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
